In [1]:
import pandas as pd
from datetime import datetime

from dotenv import load_dotenv
import os
import requests

import requests
import bs4

import psycopg2

### scrap new articles details

In [2]:
response = requests.get("https://u.today/bitcoin-news")
# soup = bs4.BeautifulSoup(response.text, 'lxml') # with lxml package
soup = bs4.BeautifulSoup(response.text, 'html.parser') 
soup

 <!DOCTYPE html>

<html class="theme-light" dir="ltr" lang="en" prefix="og: ogp.me/ns#">
<head>
<script async="" crossorigin="anonymous" src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js?client=ca-pub-9245916885225528"></script>
<link href="https://www.google-analytics.com" rel="preconnect"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=5" name="viewport"/>
<script src="/core/assets/vendor/jquery/jquery.min.js"></script>
<meta charset="utf-8">
<meta content="Latest news of Bitcoin (BTC), Bitcoin community and cryptocurrency market. Today breaking news, week reviews and month industry reports." name="description"/>
<link href="https://u.today/bitcoin-news" rel="canonical">
<meta content="article" property="og:type"/>
<meta content="https://u.today/bitcoin-news" property="og:url"/>
<meta content="Bitcoin News" property="og:title"/>
<meta content="Bitcoin News" property="og:description"/>
<meta content="https://u.today/sites/default/files/styles/16

In [3]:
# finding block about articles of the web page for the header details
article_group = soup.select("div.category__news div.news__item")
article_group

[<div class="news__item">
 <div class="news__item-head humble__row">
 <div class="humble">Apr 8, 2025 - 20:48</div>
 </div>
 <a class="news__item-body" href="https://u.today/ethbtc-somehow-keeps-plunging-even-lower">
 <div class="news__item-title">
                     ETH/BTC Somehow Keeps Plunging Even Lower                </div>
 </a>
 <div class="news__item-tags">
 <a href="/ethereum-price-prediction" hreflang="en">Ethereum Price Prediction</a>
 </div>
 </div>,
 <div class="news__item">
 <div class="news__item-head humble__row">
 <div class="humble">Apr 8, 2025 - 19:04</div>
 </div>
 <a class="news__item-body" href="https://u.today/strategy-on-cusp-of-losing-all-its-bitcoin-gains">
 <div class="news__item-title">
                     Strategy on Cusp of Losing All Its Bitcoin Gains                </div>
 </a>
 <div class="news__item-tags">
 <a href="/michael-saylor" hreflang="en">Michael Saylor</a><a href="/bitcoin-news" hreflang="en">Bitcoin News</a>
 </div>
 </div>,
 <div class="

#### limit the scrapped data to only send what we need to the PostgreSQL Table

##### check data already loaded in PosgreSQL Table hosted in Railway

In [4]:
db_conn = os.getenv("DB_CONN")
conn = psycopg2.connect(db_conn)
cur = conn.cursor()

cur.execute('''
    SELECT MAX(date)FROM utoday_article_data;
                ''')
last_utoday_article_date_PostgreSQL = cur.fetchall()
cur.close()
conn.close()

# amend the format to be a single value and not a list of one tuple
last_utoday_article_date_PostgreSQL_2=last_utoday_article_date_PostgreSQL[0][0]
last_utoday_article_date_PostgreSQL_2

datetime.datetime(2025, 4, 7, 6, 16)

In [5]:
article_date_str = []
for article in article_group:
  article_date_str.append(article.find('div', class_='humble').get_text())

print(len(article_date_str))
article_date_str

20


['Apr 8, 2025 - 20:48',
 'Apr 8, 2025 - 19:04',
 'Apr 8, 2025 - 16:43',
 'Apr 8, 2025 - 15:41',
 'Apr 8, 2025 - 15:02',
 'Apr 8, 2025 - 10:48',
 'Apr 8, 2025 - 6:30',
 'Apr 7, 2025 - 19:40',
 'Apr 7, 2025 - 14:38',
 'Apr 7, 2025 - 6:16',
 'Apr 7, 2025 - 5:21',
 'Apr 6, 2025 - 19:00',
 'Apr 6, 2025 - 14:35',
 'Apr 6, 2025 - 14:07',
 'Apr 6, 2025 - 12:30',
 'Apr 6, 2025 - 11:57',
 'Apr 6, 2025 - 9:22',
 'Apr 4, 2025 - 18:46',
 'Apr 4, 2025 - 14:45',
 'Apr 4, 2025 - 14:06']

In [6]:
article_datetime = []

for date_string in article_date_str:
    date_part = date_string.split(' - ')[0]
    time_part = date_string.split(' - ')[1]

    article_date = datetime.strptime(date_part, '%b %d, %Y').date()
    article_time = datetime.strptime(time_part, '%H:%M').time()
    combined_datetime = datetime.combine(article_date, article_time)
    article_datetime.append(combined_datetime)
article_datetime

[datetime.datetime(2025, 4, 8, 20, 48),
 datetime.datetime(2025, 4, 8, 19, 4),
 datetime.datetime(2025, 4, 8, 16, 43),
 datetime.datetime(2025, 4, 8, 15, 41),
 datetime.datetime(2025, 4, 8, 15, 2),
 datetime.datetime(2025, 4, 8, 10, 48),
 datetime.datetime(2025, 4, 8, 6, 30),
 datetime.datetime(2025, 4, 7, 19, 40),
 datetime.datetime(2025, 4, 7, 14, 38),
 datetime.datetime(2025, 4, 7, 6, 16),
 datetime.datetime(2025, 4, 7, 5, 21),
 datetime.datetime(2025, 4, 6, 19, 0),
 datetime.datetime(2025, 4, 6, 14, 35),
 datetime.datetime(2025, 4, 6, 14, 7),
 datetime.datetime(2025, 4, 6, 12, 30),
 datetime.datetime(2025, 4, 6, 11, 57),
 datetime.datetime(2025, 4, 6, 9, 22),
 datetime.datetime(2025, 4, 4, 18, 46),
 datetime.datetime(2025, 4, 4, 14, 45),
 datetime.datetime(2025, 4, 4, 14, 6)]

#### retrieve the data needed to add to the PostgreSQL Table

In [7]:
# find dates received from the API that are not in the PosgreSQL table yet
utoday_article_dates_to_append = [x for x in article_datetime if x > last_utoday_article_date_PostgreSQL_2]
print(utoday_article_dates_to_append)

[datetime.datetime(2025, 4, 8, 20, 48), datetime.datetime(2025, 4, 8, 19, 4), datetime.datetime(2025, 4, 8, 16, 43), datetime.datetime(2025, 4, 8, 15, 41), datetime.datetime(2025, 4, 8, 15, 2), datetime.datetime(2025, 4, 8, 10, 48), datetime.datetime(2025, 4, 8, 6, 30), datetime.datetime(2025, 4, 7, 19, 40), datetime.datetime(2025, 4, 7, 14, 38)]


In [8]:
N=len(utoday_article_dates_to_append)
N
# article_datetime_str = [key.strftime("%Y-%m-%d, %H:%M") for key in article_datetime]
# print(article_datetime_str)

9

In [9]:
article_datetime_str = [key.strftime("%Y-%m-%d, %H:%M") for key in utoday_article_dates_to_append]
print(article_datetime_str)

['2025-04-08, 20:48', '2025-04-08, 19:04', '2025-04-08, 16:43', '2025-04-08, 15:41', '2025-04-08, 15:02', '2025-04-08, 10:48', '2025-04-08, 06:30', '2025-04-07, 19:40', '2025-04-07, 14:38']


In [10]:
article_url = []
i = 0
while i < N:
    article_url.append(article_group[i].find('a')['href'])
    i += 1 

print(len(article_url))
article_url

9


['https://u.today/ethbtc-somehow-keeps-plunging-even-lower',
 'https://u.today/strategy-on-cusp-of-losing-all-its-bitcoin-gains',
 'https://u.today/binance-ceo-says-tariff-mayhem-may-benefit-crypto',
 'https://u.today/mega-usdt-transfer-worth-400000000-stuns-worlds-largest-crypto-exchange',
 'https://u.today/ripple-cto-responds-to-new-satoshi-nakamoto-identity-rumors',
 'https://u.today/binance-delists-14-coins-in-one-blow-market-reacts',
 'https://u.today/schiff-agrees-with-saylors-bitcoin-take',
 'https://u.today/satoshis-portfolio-collapses-by-30-billion',
 'https://u.today/breaking-xrp-collapses-8-in-minutes-because-of-this-one-word']

In [11]:
article_title = []
i = 0
while i < N:
    # article_url.append(article_group[i].find('a')['href'])
    article_title.append(article_group[i].findChild('a').get_text().strip())
    i += 1 

print(len(article_title))
article_title

9


['ETH/BTC Somehow Keeps Plunging Even Lower',
 'Strategy on Cusp of Losing All Its Bitcoin Gains',
 'Binance CEO Says Tariff Mayhem May Benefit Crypto',
 "Mega USDT Transfer Worth $400,000,000 Stuns World's Largest Crypto Exchange",
 'Ripple CTO Responds to New Satoshi Nakamoto Identity Rumors',
 'Binance Delists 14 Coins in One Blow: Market Reacts',
 'Schiff Agrees with Saylor’s Bitcoin Take',
 "Satoshi's Portfolio Collapses by $30 Billion",
 'Breaking: XRP Collapses 8% in Minutes Because of This One Word']

### assign sentiment with Hugging face

In [13]:
api_key = os.getenv("HF_API_KEY")
url = "https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
headers = { "Authorization": f"Bearer {api_key}" }

In [15]:
payload = { "inputs": article_title } # "inputs" will accept single string or list of strings

In [22]:
response = requests.post(url, headers=headers, json=payload)
response

<Response [200]>

In [23]:
sentiment_response=response.json()

In [24]:
N=len(sentiment_response)

In [26]:

# # def get_article_sentiment():
# #     # event, context
# #     load_dotenv()
# # article_title=event["responsePayload"]
# # article_title=['ETH/BTC Somehow Keeps Plunging Even Lower', 'Strategy on Cusp of Losing All Its Bitcoin Gains', 'Binance CEO Says Tariff Mayhem May Benefit Crypto', "Mega USDT Transfer Worth $400,000,000 Stuns World's Largest Crypto Exchange", 'Ripple CTO Responds to New Satoshi Nakamoto Identity Rumors', 'Binance Delists 14 Coins in One Blow: Market Reacts', 'Schiff Agrees with Saylor’s Bitcoin Take', "Satoshi's Portfolio Collapses by $30 Billion", 'Breaking: XRP Collapses 8% in Minutes Because of This One Word']

api_key = os.getenv("HF_API_KEY")
url = "https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
headers = { "Authorization": f"Bearer {api_key}" }
payload = { "inputs": article_title } # "inputs" will accept single string or list of strings
response = requests.post(url, headers=headers, json=payload)

sentiment_response=response.json()
N=len(sentiment_response)
sentiment_analysis_rows_to_append=[]
i = 0
while i < N:
    sentiment_analysis_rows_to_append.append(list(sentiment_response[i][0].items())[0][1])   
    i += 1    
# return sentiment_analysis_rows_to_append
print(sentiment_analysis_rows_to_append) # to remove for AWS
# get_article_sentiment() # to remove for AWS

['negative', 'negative', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'negative', 'negative']


### update PostgreSQL Table with article data

In [42]:
Article_df = pd.DataFrame({
    "title": article_title,
    "url": article_url,
    "date": article_datetime_str,
    "sentiment": sentiment_analysis_rows_to_append
})

# dataframe to a list of tuples
article_data_multiple_rows = list(Article_df.itertuples(index=False, name=None))
article_data_multiple_rows

[('ETH/BTC Somehow Keeps Plunging Even Lower',
  'https://u.today/ethbtc-somehow-keeps-plunging-even-lower',
  '2025-04-08, 20:48',
  'negative'),
 ('Strategy on Cusp of Losing All Its Bitcoin Gains',
  'https://u.today/strategy-on-cusp-of-losing-all-its-bitcoin-gains',
  '2025-04-08, 19:04',
  'negative'),
 ('Binance CEO Says Tariff Mayhem May Benefit Crypto',
  'https://u.today/binance-ceo-says-tariff-mayhem-may-benefit-crypto',
  '2025-04-08, 16:43',
  'positive'),
 ("Mega USDT Transfer Worth $400,000,000 Stuns World's Largest Crypto Exchange",
  'https://u.today/mega-usdt-transfer-worth-400000000-stuns-worlds-largest-crypto-exchange',
  '2025-04-08, 15:41',
  'neutral'),
 ('Ripple CTO Responds to New Satoshi Nakamoto Identity Rumors',
  'https://u.today/ripple-cto-responds-to-new-satoshi-nakamoto-identity-rumors',
  '2025-04-08, 15:02',
  'neutral'),
 ('Binance Delists 14 Coins in One Blow: Market Reacts',
  'https://u.today/binance-delists-14-coins-in-one-blow-market-reacts',
  '2

In [45]:
conn = psycopg2.connect(db_conn)
cur = conn.cursor()

In [ ]:

cur.executemany(
    '''
    INSERT INTO utoday_article_data(title, url, date, sentiment)
    VALUES (%s, %s, %s, %s);
    ''', article_data_multiple_rows)





In [47]:
conn.commit()

In [ ]:
cur.close()
conn.close()